<a href="https://colab.research.google.com/github/poojan007/ICT619-Waste-Sorting-Assistant/blob/main/ICT619-Assignment-1-Data-Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SmartSort Model
This model provides outlines a process for loading augmenting, and training a convolutional neural network (CNN) on image data for a binary classification task.

## Importing Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
import numpy as np

## Define dataset paths

In [ ]:
train_dir = '/content/data/DATASET/TRAIN'
validation_dir = '/content/data/DATASET/TEST'

## Data Augmentation

A data augmentation pipeline is created using a 'Sequential' model, which includes random flips, rotations, zooms, and translations. This is to artifically expland the training dataset by generating modifiied versions of the training images, helping the model generalize better.

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
  layers.experimental.preprocessing.RandomZoom(0.2),
  layers.experimental.preprocessing.RandomTranslation(height_factor=0.2, width_factor=0.2)
])


## Loading the datasets

The training and validation datasets are loaded from directories, specifying image size, batch size, and how the data is split. The 'validation_split' parameter

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    validation_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


## Configuring Datasets for Performance

These lines configure the datasets for performance '.cache()' keeps the images in memory after they're loaded off disk during the first epoch, '.shuffle()' randamizes the order of the images to reduce model overfitting, and '.prefetch()' overlaps data preprocessing and model execution while training.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Model Building

Building the CNN model architecture. It starts with the data augmentation layer, followed by a rescaling layer to normalize pixel values. Then, it adds convolutional and max pooling layers for feature extraction, followed by dense layers for classification. The final layer uses a sigmoid activation function suitable for binary classification.

In [ ]:
model = models.Sequential([
    data_augmentation,
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


## Compile the Model

The model is compiled with the Adam optimizer and binary crossentropy loss function, which is appropriate for binary classification tasks. The model's performance will be evaluated based on accuracy.

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Model Training

Finally, the model is trained for a defined number of epochs using the training dataset, with validation performed on the validation dataset. The 'fit' method returns a history object containing training and validation loss and accuracy for each epoch, which can be used for analysis of the model's performance over time.

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


## Validating the model

In [ ]:
# Function to prepare and predict the class of an image
def predict_with_augmentation(img_path, model):
    # Load the image
    img = image.load_img(img_path, target_size=(img_height, img_width))

    # Convert the image to a numpy array
    img_array = image.img_to_array(img)

    # Expand dimensions to match the shape of model input
    img_array = np.expand_dims(img_array, axis=0)

    # Predict (data augmentation is applied automatically)
    prediction = model.predict(img_array)

    # Assuming binary classification with a sigmoid activation, adjust if necessary
    predicted_class = 'Class 1' if prediction[0] > 0.5 else 'Class 0'

    return predicted_class

# Example usage
img_path = 'path/to/your/image.jpg'
predicted_class = predict_with_augmentation(img_path, model)
print(f'Predicted class: {predicted_class}')